# Рекомендация тарифов

В вашем распоряжении данные о поведении клиентов, которые уже перешли на эти тарифы (из проекта курса «Статистический анализ данных»). Нужно построить модель для задачи классификации, которая выберет подходящий тариф. Предобработка данных не понадобится — вы её уже сделали.

Постройте модель с максимально большим значением *accuracy*. Чтобы сдать проект успешно, нужно довести долю правильных ответов по крайней мере до 0.75. Проверьте *accuracy* на тестовой выборке самостоятельно.

## Откройте и изучите файл

In [1]:
import pandas as pd #для обработки данных
from sklearn.model_selection import train_test_split #для деления набора данных 
from sklearn.tree import DecisionTreeClassifier #импортируем дерево решений
from sklearn.ensemble import RandomForestClassifier #модель случайного леса
from sklearn.linear_model import LogisticRegression #модель логистической регрессии
!pip install tabulate
from tabulate import tabulate #для отображения табличных данных
from sklearn.dummy import DummyClassifier #для адекватности

In [2]:
data = pd.read_csv("/datasets/users_behavior.csv") #открываем файл
display(data.head(5)) #вывод первых 5 строк
print(data.info()) #информация о датасете

,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     3214 non-null   float64
 1   minutes   3214 non-null   float64
 2   messages  3214 non-null   float64
 3   mb_used   3214 non-null   float64
 4   is_ultra  3214 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 125.7 KB
None


Описание данных

* calls - количество звонков
* minutes - суммарное количество минут
* messages - количество сообщений
* mb_used  -  количество трафика в МБ
* is_ultra - при 1 тариф ультра, при 0 смарт

На данном этапе, все задачи выполнены, так как предобработка данных уже была проведена. Стоит лишь отметить, что перед нами классическая задача классификации -  опредить подходящий тариф на основе эти данных.

## Разбейте данные на выборки

Так как спрятанной тестовой выборки нет, предлагаю поделить датасет на 60-20-20. Сначала отдадим 60% обучающей, 40% тестовой

In [3]:
data_train, data_test = train_test_split(data, test_size = 0.4, random_state = 12345) # закрепляем шаг для повторения эксперемента
#на первом этапе размер валидационной выборки 0.6, тестосовой 0.4

Создаем признаки и целевой признак на тестовом наборе данных

In [4]:
features = data_test.drop(["is_ultra"], axis = 1) #признаки
target =  data_test["is_ultra"] #целевой признак

Делим тестовую выборку пополам, чтобы получить валидационную и тестовые выборки.

In [5]:
features_valid, features_test, target_valid, target_test = train_test_split(features, target, test_size= 0.5 , random_state=12345)
#делим оставшую часть пополам, сохраняем шаг для закрепления эксперимента

Создаем тестовую выборку

In [6]:
features_train = data_train.drop(["is_ultra"], axis = 1)
target_train = data_train["is_ultra"]

* features_train, target_train - тренировачные даннные
* features_test, target_test - тестовые даннные
* features_valid, target_valid - валидационные даннные

## Исследуйте модели

### Дерево решений

In [7]:
best_accuracy = 0 #лучшее значение
best_depth = 0 #лучшая глубина 
for depth in range(1,100): #цикл
    model = DecisionTreeClassifier(random_state = 12345, max_depth = depth) #создание модели
    model.fit(features_train, target_train) #обучение
    accuracy = model.score(features_valid, target_valid) #качество
    if accuracy > best_accuracy: #цикл для выбора лучшей модели
        best_accuracy = accuracy
        best_depth = depth
print(f'При глубине дерева - {best_depth} , качество  - {best_accuracy}')

При глубине дерева - 3 , качество  - 0.7853810264385692


### Случайный лес

In [8]:
best_accuracy = 0
best_n_estimators = 0
best_depth = 0 
for est in range(1, 50, 3):
    for depth in range(1, 100):
        model = RandomForestClassifier(random_state = 12345, n_estimators=est, max_depth = depth)
        model.fit(features_train, target_train)
        accuracy = model.score(features_valid, target_valid)
        if accuracy > best_accuracy:
            best_accuracy = accuracy
            best_n_estimators = est
            best_depth = depth
print(f'При Количестве деревьев - {best_n_estimators} , при  глубине - {best_depth}, качество  - {best_accuracy}')

При Количестве деревьев - 40 , при  глубине - 8, качество  - 0.8087091757387247


### Логистическая регрессия

In [9]:
model = LogisticRegression(random_state = 12345, solver = "lbfgs")
model.fit(features_train, target_train)
accuracy = model.score(features_valid, target_valid)
print("Качество при логистической регрессии - ", accuracy)

Качество при логистической регрессии -  0.7107309486780715


Таким образом лучшее качество достигается при использовании модели случайного леса, проверим это на тестовой выборке.

## Проверьте модель на тестовой выборке

### Дерево решений

In [10]:
model = DecisionTreeClassifier(random_state = 12345, max_depth = 3)
model.fit(features_train, target_train)
accuracy = model.score(features_test, target_test)
print("Качество - ", accuracy)

Качество -  0.7791601866251944


### Случайный лес

In [11]:
model = RandomForestClassifier(random_state = 12345, n_estimators = 40, max_depth = 8)
model.fit(features_train, target_train)
accuracy = model.score(features_test, target_test)
print("Качество - ", accuracy)

Качество -  0.7962674961119751


### Логистическая регрессия

In [12]:
model = LogisticRegression(random_state = 12345, solver = "lbfgs")
model.fit(features_train, target_train)
accuracy = model.score(features_test, target_test)
print("Качество при логистической регрессии - ", accuracy)

Качество при логистической регрессии -  0.6842923794712286


In [13]:
table = {'дерево_решений' : 0.77, 'случайный_лес' : 0.79, 'логистическая_регрессия' : 0.68}
print(tabulate(table.items(), headers=['Модель', 'Качество'], tablefmt="grid"))

+-------------------------+------------+
| Модель                  |   Качество |
+=========================+============+
| дерево_решений          |       0.77 |
+-------------------------+------------+
| случайный_лес           |       0.79 |
+-------------------------+------------+
| логистическая_регрессия |       0.68 |
+-------------------------+------------+


### Вывод

Таким образом, наша гипотеза была верна, модель случаного леса при определенных параметрах, показывает результаты выше чем у остальных моделей. Ее значение большое 0,75 следовательно задача в рамках проекта была выполнена.

## (бонус) Проверьте модели на адекватность

In [14]:
x = DummyClassifier(random_state = 12345, strategy = "stratified")
model.fit(features_train, target_train)
accuracy = model.score(features_test, target_test)
print("Качество при стратифицированной стратегии:", accuracy)
x = DummyClassifier(random_state = 12345, strategy = "most_frequent")
model.fit(features_train, target_train)
accuracy = model.score(features_test, target_test)
print("Качество при выборе наиболее частой метки:", accuracy )

Качество при стратифицированной стратегии: 0.6842923794712286
Качество при выборе наиболее частой метки: 0.6842923794712286
